In [1]:
from pyspark.sql.functions import *

In [2]:
hivepath="hdfs://machicura.colbunsa.cl:8020/user/hive/warehouse/"
db="bcene2019"

In [3]:
dfb=spark.read.format("parquet").load(hivepath+db+".db/plpbar")

In [4]:
dfe=spark.read.format("parquet").load(hivepath+db+".db/plpeta")

In [5]:
#join between plpbar and plpeta
dft1=dfb.join(dfe.select("BloqueE","TipoBlo","Duracion","Fecha","Categoria"), dfb.Bloque==dfe.BloqueE,"inner").drop(dfe.BloqueE)

In [6]:
#Create a column with product between Duracion and CMgBar named cruce, clean space in BarNom
dft2=dft1.select("Fecha","Hidro","Categoria","Duracion", "BarNom", "CMgBar","DemBarE","User","DateStudy","NameStudy")
dft2=dft2.withColumn("cruce",col("CMgBar")*col("Duracion")).withColumn("BarNom",trim(col("BarNom")))
dft2=dft2.withColumn("Anio",year(dft2.Fecha))

In [7]:
#group by fecha,hidro,barnom,user,datestudy and namestudy making aggregatino funcion over Duracion, DemBarE and cruce
#also, was created a column with division between cruce and Duracion
dft3=dft2.groupBy("Fecha","Categoria","Duracion","Hidro","BarNom","User","DateStudy","NameStudy","Anio").agg({'Duracion':'sum','cruce':'sum','DemBarE':'sum'})\
         .withColumn("CMg",col("sum(cruce)")/col("sum(Duracion)"))

In [12]:
dft4=dft3.select("Fecha","Categoria","Duracion","Hidro","BarNom","CMg","sum(DemBarE)","User","DateStudy","NameStudy","Anio")
dft4=dft4.withColumnRenamed("sum(DemBarE)","Demanda")#.withColumn("Fecha",to_date(col("Fecha")).cast("date"))

In [13]:
dft4.printSchema()
dft4.show(4)

root
 |-- Fecha: timestamp (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Duracion: decimal(3,0) (nullable = true)
 |-- Hidro: string (nullable = true)
 |-- BarNom: string (nullable = true)
 |-- CMg: double (nullable = true)
 |-- Demanda: double (nullable = true)
 |-- User: string (nullable = true)
 |-- DateStudy: timestamp (nullable = true)
 |-- NameStudy: string (nullable = true)
 |-- Anio: integer (nullable = true)

+-------------------+------------+--------+------+-----------+-----+-------+------+-------------------+---------+----+
|              Fecha|   Categoria|Duracion| Hidro|     BarNom|  CMg|Demanda|  User|          DateStudy|NameStudy|Anio|
+-------------------+------------+--------+------+-----------+-----+-------+------+-------------------+---------+----+
|2020-07-01 00:00:00|Madrugada   |      62|Sim 51|polpaico500|45.39|    0.0|jaraya|2019-02-01 00:00:00|       cb|2020|
|2021-02-01 00:00:00|Dia         |     114|Sim 51|polpaico500|48.52|    0.0|jaraya|2

In [ ]:
dft4.write.format("parquet").partitionBy("Hidro").saveAsTable(db+".plpbarsl")